# GRAD-E1326: Python Programming for Data Scientists
## Ph.D. Hannah Béchara
### Ji Yoon Han & Mariana G. Carrillo 

**Initial Project report: Tweet Sentiment Analysis**


In [ ]:
#Importing libraries for sentiment analysis 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk #Natural Language Processing Package 
import os #functions for interacting with the operating system
# import spacy #Models for NLP
# import torch #also for NLP
from tqdm.notebook import tqdm 
# from transformers import BertTokenizer
# from torch.utils.data import TensorDataset
# import transformers #contains pretrained models to perform tasks on texts
# from transformers import BertForSequenceClassification
# from wordcloud import WordCloud #For nice wordclouds
# import tensorflow as tf #Package to develop train models 
# from tensorflow.keras.preprocessing import text 
# from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.layers import *
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.utils import to_categorical
from nltk.corpus import stopwords, words
from nltk.stem import WordNetLemmatizer
import time #for handling dates and times
import re 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.metrics import AUC
from sklearn.metrics import confusion_matrix, classification_report
from nltk.tokenize import TweetTokenizer


Loading data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading and cleaning data (for Google Colab)
from google.colab import files 
uploaded = files.upload()

In [ ]:
import io

train_data = pd.read_csv(io.BytesIO(uploaded['Corona_NLP_train.csv']), encoding = 'latin-1')
test_data = pd.read_csv(io.BytesIO(uploaded['Corona_NLP_test.csv']), encoding = 'latin-1')

In [ ]:
# # Loading and cleaning data
# train_data = pd.read_csv('Corona_NLP_train.csv', encoding='latin-1')
# test_data = pd.read_csv('Corona_NLP_test.csv', encoding='latin-1')

**Data cleaning**

In [ ]:
# #Clean Tweets --> TRAIN DATA (these functions run locally, not on jupiter notebooks)

# !pip3 install tweet-preprocessor #processing library 
# # !pip3 install preprocessor
# import preprocessor as p 

# # def preprocess_tweet(row):
# #     text = row['OriginalTweet']
# #     text = p.clean(text)
# #     return text

# train_data['OriginalTweet'] = train_data.apply(preprocess_tweet, axis=1) #remove urls, hashtags, mentions, etc.

# train_data.count() #check tweets

# train_data = train_data.dropna() #drop NAs
# train_data = train_data.drop_duplicates() #drop duplicates

# train_data.count() #check tweets again

# from gensim.parsing.preprocessing import remove_stopwords #removing stopwords

# def stopword_removal(row):
#     text = row['OriginalTweet']
#     text = remove_stopwords(text)
#     return text

# test_data['OriginalTweet'] = test_data.apply(stopword_removal, axis=1)

# test_data['OriginalTweet'] = test_data['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ') #make lowercase, remove punctuation

# test_data.head() #check it worked

In [ ]:
!python --version

In [ ]:
!pip3 install tweet-preprocessor
import preprocessor as p

# p.clean('test test https://allisonkoh.github.io/')

In [ ]:
#Clean Tweets --> TRAIN DATA

def preprocess_tweet(row):
    text = row['OriginalTweet']
    text = p.clean(text)
    return text

train_data['OriginalTweet'] = train_data.apply(preprocess_tweet, axis=1) #remove urls, hashtags, mentions, etc.

train_data.count() #check tweets

train_data = train_data.dropna() #drop NAs
train_data = train_data.drop_duplicates() #drop duplicates

train_data.count() #check tweets again

from gensim.parsing.preprocessing import remove_stopwords #removing stopwords

def stopword_removal(row):
    text = row['OriginalTweet']
    text = remove_stopwords(text)
    return text

train_data['OriginalTweet'] = train_data.apply(stopword_removal, axis=1)

train_data['OriginalTweet'] = train_data['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ') #make lowercase, remove punctuation

train_data.head()

In [ ]:
#Clean Tweets --> TEST DATA

def preprocess_tweet(row):
    text = row['OriginalTweet']
    text = p.clean(text)
    return text

test_data['OriginalTweet'] = test_data.apply(preprocess_tweet, axis=1) #remove urls, hashtags, mentions, etc.

test_data.count() #check tweets

test_data = test_data.dropna() #drop NAs
test_data = test_data.drop_duplicates() #drop duplicates

test_data.count() #check tweets again

from gensim.parsing.preprocessing import remove_stopwords #removing stopwords

def stopword_removal(row):
    text = row['OriginalTweet']
    text = remove_stopwords(text)
    return text

test_data['OriginalTweet'] = test_data.apply(stopword_removal, axis=1)

test_data['OriginalTweet'] = test_data['OriginalTweet'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ') #make lowercase, remove punctuation

test_data.head()

Data preview

In [ ]:
#Train data 

train_data.head(5) #preview
train_data.describe() #descriptive statistics


In [ ]:
#Test data

test_data.head(5) #preview
test_data.describe() #descriptive statistics


### Exploratory Data analysis

#### Test data

In [ ]:
#Create histogram --> Distribution TEST DATA
#Can we also make this a function?
plt.figure(figsize=(12,6)) #specifying the size of the figure
sns.set_palette("Spectral") #color palette
sns.countplot(x='Sentiment', data=test_data, order=['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive'], )
plt.xlabel('Sentiment(tag)')
plt.ylabel('Count of tweets')
plt.suptitle('Histogram of tweet distribution per sentiment classification (Test data)')


In [ ]:
#Distribution of tweet counts 
test_data.groupby(['TweetAt', 'Sentiment'])['OriginalTweet'].count().unstack().plot(kind='line', figsize=(12, 6))
plt.title('Tweets on Coronavirus March 2020 (Test data)')
plt.ylabel('Tweet Count')

In [ ]:
#Attempt to create wordcloud - test data
def wordcloud2(test_data):
    stopwords = set(STOPWORDS)
    stopwords.add("https")
    stopwords.add("00A0")
    stopwords.add("00BD")
    stopwords.add("00B8")
    stopwords.add("ed")
    wordcloud2 = WordCloud(background_color="white",stopwords=stopwords).generate(" ".join([i for i in test_data['OriginalTweet'].str.upper()]))
    plt.imshow(wordcloud2)
    plt.axis("off")
    plt.title("Most common words, test data")
    figsize=(12, 6)

wordcloud2(test_data)  

#### Train data

In [ ]:
#Create histogram --> Distribution TRAIN DATA
plt.figure(figsize=(12,6))
sns.set_palette("Spectral")
sns.countplot(x='Sentiment', data=train_data, order=['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive'], )
plt.xlabel('Sentiment (tag)')
plt.ylabel('Count of tweets')
plt.suptitle('Histogram of tweet distribution per sentiment classification (Train data)')

In [ ]:
#Distribution of tweet counts 
train_data.groupby(['TweetAt', 'Sentiment'])['OriginalTweet'].count().unstack().plot(kind='line', figsize=(12, 6))
plt.title('Tweets on Coronavirus, March 2020, (Train data)')
plt.ylabel('Tweet Count')

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

#Attempt to create wordcloud
def wordcloud1(training_data):
    stopwords = set(STOPWORDS)
    stopwords.add("https")
    stopwords.add("00A0")
    stopwords.add("00BD")
    stopwords.add("00B8")
    stopwords.add("ed")
    wordcloud1 = WordCloud(background_color="white",stopwords=stopwords).generate(" ".join([i for i in train_data['OriginalTweet'].str.upper()]))
    plt.imshow(wordcloud1)
    plt.axis("off")
    plt.title("Most common words, training data")
    figsize=(12, 6)

wordcloud1(train_data)  

In [ ]:

# Import nltk / stopwords
import nltk
nltk.download('stopwords')

# Define stopwords 
stop_words = stopwords.words('english') #defining var to remove stopwords in the process_tweet function 

# Define function for cleaning tweets 
def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', ' ', tweet) #removing urls
    tweet = re.sub(r'<.*?>', ' ', tweet)  # removing html tags    
    tweet = re.sub(r'\d+', ' ', tweet) #removing digits
    tweet = re.sub(r'#\w+', ' ', tweet)    #removing hashtags
    tweet = re.sub(r'@\w+', ' ', tweet) #removing mentions
    tweet = tweet.split() #removing stop words
    tweet = " ".join([word for word in tweet if not word in stop_words])
    return tweet

In [ ]:
# Clean tweets from train data by creating a new column in the train_data df
train_data['CleanTweet'] = train_data['OriginalTweet'].apply(lambda x: clean_tweet(x))
train_data.head(10)

In [ ]:
# Clean tweets from test data by creating a new column in the test_data df
test_data['CleanTweet'] = test_data['OriginalTweet'].apply(lambda x: clean_tweet(x))
test_data.head(10)

### Tokenization

In [ ]:
# Comparing different tokenizers (https://towardsdatascience.com/an-introduction-to-tweettokenizer-for-processing-tweets-9879389f8fe7)
# NOTE: Once the tweets are clean, the tokenizers might not be that different.
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer 
# from nltk.tokenize import RegrexTokenizer
from nltk.tokenize import TweetTokenizer 
from nltk.tokenize import RegexpTokenizer

# Sample clean tweets 
compare_list = train_data['CleanTweet'].head(10)

# Code for using each tokenizer 

## separate words using spaces and punctuations

word_tokens = []
for sent in compare_list:
    print(word_tokenize(sent))
    word_tokens.append(word_tokenize(sent))

## split all punctuations into separate tokens

punct_tokenizer = WordPunctTokenizer()

punct_tokens = []
for sent in compare_list:
    print(punct_tokenizer.tokenize(sent))
    punct_tokens.append(punct_tokenizer.tokenize(sent))

## tweet tokenizer 

tweet_tokenizer = TweetTokenizer()

tweet_tokens = []
for sent in compare_list:
    print(tweet_tokenizer.tokenize(sent))
    tweet_tokens.append(tweet_tokenizer.tokenize(sent))

## Match on whitespace

space_tokenizer = RegexpTokenizer("\s+", gaps=True)

space_tokens = []
for sent in compare_list:
    
    print(space_tokenizer.tokenize(sent))
    space_tokens.append(space_tokenizer.tokenize(sent))


# Compare tokenizers 

tokenizers = {'word_tokenize': word_tokens,
             'WordPunctTokenize':punct_tokens,
            #  'RegrexTokenizer for matching':match_tokens,
             'RegrexTokenizer for white space': space_tokens,
             'TweetTokenizer': tweet_tokens }

df_comparing_tokenizers = pd.DataFrame.from_dict(tokenizers)

df_comparing_tokenizers.head(10)

In [ ]:
# Writing a function once you choose a tokenizer (pseudo code below; insert your tokenizer of choice)

# def tokenize_tweet(tweet):
#   tweet = word_tokenize(tweet) # replace word_tokenize with your tokenizer of choice if you want to use a different one.
#   return tweet

# test_data['TokenizedTweet'] = test_data['CleanTweet'].apply(lambda x: tokenize_tweet(x))

### References 
* Matplotlib.org. 2020. Pyplot Tutorial — Matplotlib 3.3.2 Documentation. [online] Available at: <https://matplotlib.org/tutorials/introductory/pyplot.html> [Accessed 20 October 2020].
* Kaggle.com. 2020. Sentiment Prediction. [online] Available at: <https://www.kaggle.com/shahraizanwar/covid19-tweets-sentiment-prediction-rnn-85-acc> [Accessed 18 October 2020].
* ***Towards Data Science. How to Tokenize Tweets with Python https://towardsdatascience.com/an-introduction-to-tweettokenizer-for-processing-tweets-9879389f8fe7***